In [1]:
import pandas as pd
from sklearn.metrics import accuracy_score

import warnings
warnings.filterwarnings('ignore')

In [2]:
df_12 = pd.read_csv('data/actuals_predictions_state_2012.csv', index_col=0)
df_16 = pd.read_csv('data/actuals_predictions_state_2016.csv', index_col=0)
df_20 = pd.read_csv('data/actuals_predictions_state_2020.csv', index_col=0)

In [3]:
df_20['TOP_TOPIC'].unique()

array(['Economy', 'Climate Change', 'Healthcare', 'Immigration'],
      dtype=object)

In [4]:
df_12.dtypes

STATE_FIPS              int64
PRED_DEM_VOTES          int64
PRED_REP_VOTES          int64
PRED_WINNING_PARTY      int64
STATE                  object
TOTAL_DEM_VOTES        object
TOTAL_REP_VOTES        object
TOTAL_VOTES            object
MARGIN_VICTORY        float64
ABS_MARGIN_VICTORY    float64
WINNING_PARTY           int64
ELECTORAL_VOTES         int64
TOP_TOPIC              object
dtype: object

In [15]:
df_12[["PRED_DEM_VOTES", "PRED_REP_VOTES"]] = df_12[["PRED_DEM_VOTES", "PRED_REP_VOTES"]].apply(pd.to_numeric)
df_16[["PRED_DEM_VOTES", "PRED_REP_VOTES"]] = df_16[["PRED_DEM_VOTES", "PRED_REP_VOTES"]].apply(pd.to_numeric)
df_20[["PRED_DEM_VOTES", "PRED_REP_VOTES"]] = df_20[["PRED_DEM_VOTES", "PRED_REP_VOTES"]].apply(pd.to_numeric)

df_12['PRED_TOTAL_VOTES'] = df_12['PRED_DEM_VOTES'] + df_12['PRED_REP_VOTES']
df_16['PRED_TOTAL_VOTES'] = df_16['PRED_DEM_VOTES'] + df_16['PRED_REP_VOTES']
df_20['PRED_TOTAL_VOTES'] = df_20['PRED_DEM_VOTES'] + df_20['PRED_REP_VOTES']

In [16]:
def get_winner_by_topic_and_year(tp, yr):
    if tp == 'Economy':
        if yr == '2012':
            return 'r'
        if yr == '2016':
            return 'r'
        if yr == '2020':
            return 'd'
    elif tp == 'Immigration':
        if yr == '2012':
            return 'd'
        if yr == '2016':
            return 'd'
        if yr == '2020':
            return 'd'
    elif tp == 'Healthcare':
        if yr == '2012':
            return 'r'
        if yr == '2016':
            return 'd'
        if yr == '2020':
            return 'd'
    elif tp == 'Climate Change':
        if yr == '2012':
            return 'd'
        if yr == '2016':
            return 'd'
        if yr == '2020':
            return 'd'

In [21]:
#TODO - Get the new winning margin

def adjust_weights_and_get_acc(df, weight, yr, isValidation):

    df2 = df[['STATE_FIPS', 'STATE', 'WINNING_PARTY', 'PRED_WINNING_PARTY', 'PRED_DEM_VOTES', 'PRED_REP_VOTES']]
    
    acc_before = 0
    
    if isValidation:
        acc_before = accuracy_score(df2['WINNING_PARTY'], df2['PRED_WINNING_PARTY'])

    for idx, row in df.iterrows():
        dem_votes = 0
        rep_votes = 0

        if get_winner_by_topic_and_year(row['TOP_TOPIC'], yr) == 'd':
            dem_votes = row['PRED_DEM_VOTES'] + (row['PRED_TOTAL_VOTES'] * weight)
            rep_votes = row['PRED_REP_VOTES'] - (row['PRED_TOTAL_VOTES'] * weight)
        else:
            dem_votes = row['PRED_DEM_VOTES'] - (row['PRED_TOTAL_VOTES'] * weight)
            rep_votes = row['PRED_REP_VOTES'] + (row['PRED_TOTAL_VOTES'] * weight)

        df2.loc[idx,'ADJUSTED_DEM_VOTES'] = dem_votes
        df2.loc[idx,'ADJUSTED_REP_VOTES'] = rep_votes
            
        if dem_votes > rep_votes and row['PRED_WINNING_PARTY'] == 1:
            print('rep->dem', row['STATE'])
            df2.loc[idx,'ADJUSTED_WINNING_PARTY'] = 0
            df2.loc[idx,'WINNER_SHIFT'] = 'REP->DEM'
        elif dem_votes < rep_votes and row['PRED_WINNING_PARTY'] == 0:
            print('dem->rep', row['STATE'])
            df2.loc[idx,'ADJUSTED_WINNING_PARTY'] = 1
            df2.loc[idx,'WINNER_SHIFT'] = 'DEM->REP'
        else:
            df2.loc[idx,'ADJUSTED_WINNING_PARTY'] = row['PRED_WINNING_PARTY']
 
    acc_after = 0
    
    if isValidation:
        acc_after = accuracy_score(df2['WINNING_PARTY'], df2['ADJUSTED_WINNING_PARTY'])

    
    return acc_before, acc_after, df2

In [22]:
start = 0.025
acc_before, acc_after, data = adjust_weights_and_get_acc(df_12, start, '2012', True)
print(acc_before, acc_after)
#data.head(50)

data.to_csv('data/predictions_vs_adjusted_2012.csv')

#for factor in range(100):
    
#    ab, af = adjust_weights_and_get_acc(df_12, start)
#    start = start + 0.0013
    
   # print(af - ab)

#adjust_weights_and_get_acc(df_12)


dem->rep Alaska
dem->rep Ohio
dem->rep Oregon
dem->rep Texas
0.8823529411764706 0.8823529411764706


In [23]:

acc_before, acc_after, data = adjust_weights_and_get_acc(df_16, start, '2016', True)
print(acc_before, acc_after)
data.to_csv('data/predictions_vs_adjusted_2016.csv')
#data.head(50)

dem->rep Georgia
dem->rep Ohio
dem->rep Texas
rep->dem Virginia
0.8235294117647058 0.9019607843137255


In [27]:
print(df_20.columns)
acc_before, acc_after, data = adjust_weights_and_get_acc(df_20, start, '2020', False)
print(acc_before, acc_after)
data.to_csv('data/predictions_vs_adjusted_2020.csv')
data.head(50)

Index(['STATE_FIPS', 'PRED_DEM_VOTES', 'PRED_REP_VOTES', 'PRED_WINNING_PARTY',
       'STATE', 'TOTAL_DEM_VOTES', 'TOTAL_REP_VOTES', 'TOTAL_VOTES',
       'MARGIN_VICTORY', 'ABS_MARGIN_VICTORY', 'WINNING_PARTY',
       'ELECTORAL_VOTES', 'TOP_TOPIC', 'PRED_TOTAL_VOTES'],
      dtype='object')
rep->dem Alabama
rep->dem Alabama
rep->dem Louisiana
rep->dem Maine
rep->dem Missouri
rep->dem North Carolina
rep->dem Pennsylvania
rep->dem South Carolina
rep->dem Virginia
0 0


,STATE_FIPS,STATE,WINNING_PARTY,PRED_WINNING_PARTY,PRED_DEM_VOTES,PRED_REP_VOTES,ADJUSTED_DEM_VOTES,ADJUSTED_REP_VOTES,ADJUSTED_WINNING_PARTY,WINNER_SHIFT
0,1,Alabama,NaN,1,1037054,1083735,1.090074e+06,1030715.275,0.0,REP->DEM
1,1,Alabama,NaN,1,1037054,1083735,1.090074e+06,1030715.275,0.0,REP->DEM
2,2,Alaska,NaN,0,242744,223455,2.543990e+05,211800.025,0.0,NaN
3,2,Alaska,NaN,0,242744,223455,2.543990e+05,211800.025,0.0,NaN
4,4,Arizona,NaN,0,1779734,820395,1.844737e+06,755391.775,0.0,NaN
5,4,Arizona,NaN,0,1779734,820395,1.844737e+06,755391.775,0.0,NaN
6,5,Arkansas,NaN,1,504135,715696,5.346308e+05,685200.225,1.0,NaN
7,5,Arkansas,NaN,1,504135,715696,5.346308e+05,685200.225,1.0,NaN
8,6,California,NaN,0,10310902,3766648,1.066284e+07,3414709.250,0.0,NaN
9,6,California,NaN,0,10310902,3766648,1.066284e+07,3414709.250,0.0,NaN
